In [1]:
import numpy as np
from pokerkit import (
    Automation,
    FixedLimitTexasHoldem,
)
import tqdm

In [2]:
# Create a game template (factory) that we can reuse
game = FixedLimitTexasHoldem(
    automations=(
        Automation.ANTE_POSTING,
        Automation.BET_COLLECTION,
        Automation.BLIND_OR_STRADDLE_POSTING,
        Automation.CARD_BURNING,
        Automation.HOLE_DEALING,
        Automation.BOARD_DEALING,
        Automation.HOLE_CARDS_SHOWING_OR_MUCKING,
        Automation.HAND_KILLING,
        Automation.CHIPS_PUSHING,
        Automation.CHIPS_PULLING,
    ),
    ante_trimming_status=True,  # use blinds instead of antes
    raw_antes=0,
    raw_blinds_or_straddles=(2, 4),
    small_bet=4,
    big_bet=8,
    starting_board_count=1,
)

# Create initial state by calling the game with stacks and player count
state = game(
    raw_starting_stacks=200,
    player_count=5,  # the agent plus 4 opponents - one of each archetype
)
state

State(automations=(<Automation.ANTE_POSTING: 'Ante posting'>, <Automation.BET_COLLECTION: 'Bet collection'>, <Automation.BLIND_OR_STRADDLE_POSTING: 'Blind or straddle posting'>, <Automation.CARD_BURNING: 'Card burning'>, <Automation.HOLE_DEALING: 'Hole dealing'>, <Automation.BOARD_DEALING: 'Board dealing'>, <Automation.HOLE_CARDS_SHOWING_OR_MUCKING: 'Hole cards showing or mucking'>, <Automation.HAND_KILLING: 'Hand killing'>, <Automation.CHIPS_PUSHING: 'Chips pushing'>, <Automation.CHIPS_PULLING: 'Chips pulling'>), deck=<Deck.STANDARD: (2c, 2d, 2h, 2s, 3c, 3d, 3h, 3s, 4c, 4d, 4h, 4s, 5c, 5d, 5h, 5s, 6c, 6d, 6h, 6s, 7c, 7d, 7h, 7s, 8c, 8d, 8h, 8s, 9c, 9d, 9h, 9s, Tc, Td, Th, Ts, Jc, Jd, Jh, Js, Qc, Qd, Qh, Qs, Kc, Kd, Kh, Ks, Ac, Ad, Ah, As)>, hand_types=(<class 'pokerkit.hands.StandardHighHand'>,), streets=(Street(card_burning_status=False, hole_dealing_statuses=(False, False), board_dealing_count=0, draw_status=False, opening=<Opening.POSITION: 'Position'>, min_completion_betting_or_ra

In [10]:
# Simulate 10 hands with random actions
num_hands = 10

for hand_num in tqdm.tqdm(range(num_hands), desc="Simulating hands"):
    print(f"\n{'='*60}")
    print(f"HAND {hand_num + 1}")
    print(f"{'='*60}")
    
    # Create a fresh state using the game factory
    state = game(
        raw_starting_stacks=200,
        player_count=5,
    )
    
    print(f"Starting stacks: {state.stacks}")
    print(f"\n--- HOLE CARDS ---")
    for i, hole_cards in enumerate(state.hole_cards):
        print(f"Player {i}: {hole_cards}")
    
    print(f"\n--- GAME PLAY ---")
    action_count = 0
    
    # Play the hand with random actions
    while state.status:
        # Track board state changes
        current_board = state.board_cards[0] if state.board_cards else []
        board_label = ""
        if len(current_board) == 3:
            board_label = "FLOP"
        elif len(current_board) == 4:
            board_label = "TURN"
        elif len(current_board) == 5:
            board_label = "RIVER"
        
        if board_label and action_count > 0:
            print(f"\n{board_label}: {current_board}")
        
        # Check what actions are available
        if state.actor_indices:  # If there's an active player who needs to act
            available_actions = []
            
            # Check if folding is possible
            if state.can_fold():
                available_actions.append('fold')
            
            # Check if checking/calling is possible
            if state.can_check_or_call():
                available_actions.append('check_or_call')
            
            # Check if betting/raising is possible
            if state.can_complete_bet_or_raise_to():
                available_actions.append('bet_or_raise')
            
            # Choose a random action
            if available_actions:
                action = np.random.choice(available_actions)
                actor = state.actor_indices[0]
                
                # Print the action
                action_str = ""
                if action == 'fold':
                    state.fold()
                    action_str = "folds"
                elif action == 'check_or_call':
                    call_amount = state.checking_or_calling_amount
                    state.check_or_call()
                    action_str = "checks" if call_amount == 0 else f"calls {call_amount}"
                elif action == 'bet_or_raise':
                    min_amount = state.min_completion_betting_or_raising_to_amount
                    state.complete_bet_or_raise_to()
                    action_str = f"bets/raises to {min_amount}"
                
                print(f"Player {actor} (position {actor}): {action_str} [Pot: {state.total_pot_amount}]")
                action_count += 1
        else:
            # No player actions needed, state will auto-advance
            break
    
    print(f"\n--- FINAL BOARD ---")
    if len(state.board_cards) > 0:
        final_board = " ".join([f"{str(card[0].rank)}{str(card[0].suit)}" for card in state.board_cards])
    else:
        final_board = "EMPTY"
    print(f"Board: {final_board}")
    
    print(f"\n--- RESULTS ---")
    print(f"Final stacks: {state.stacks}")
    print(f"Payoffs: {state.payoffs}")
    
    # Show winning hands
    print(f"\nActive players at showdown:")
    for i, is_active in enumerate(state.statuses):
        if is_active:
            print(f"  Player {i}: {state.hole_cards[i]}")

Simulating hands: 100%|██████████| 10/10 [00:00<00:00, 503.96it/s]


HAND 1
Starting stacks: [198, 196, 200, 200, 200]

--- HOLE CARDS ---
Player 0: [Qd, Jc]
Player 1: [Kd, 2h]
Player 2: [5h, 2c]
Player 3: [8h, Kc]
Player 4: [3h, 7s]

--- GAME PLAY ---
Player 2 (position 2): folds [Pot: 6]
Player 3 (position 3): folds [Pot: 6]
Player 4 (position 4): bets/raises to 8 [Pot: 14]
Player 0 (position 0): bets/raises to 12 [Pot: 24]
Player 1 (position 1): folds [Pot: 24]
Player 4 (position 4): folds [Pot: 0]

--- FINAL BOARD ---
Board: EMPTY

--- RESULTS ---
Final stacks: [212, 196, 200, 200, 192]
Payoffs: [12, -4, 0, 0, -8]

Active players at showdown:
  Player 0: [Qd, Jc]

HAND 2
Starting stacks: [198, 196, 200, 200, 200]

--- HOLE CARDS ---
Player 0: [Ts, 3c]
Player 1: [3d, Kh]
Player 2: [9c, 4c]
Player 3: [Td, 9d]
Player 4: [2h, 5d]

--- GAME PLAY ---
Player 2 (position 2): folds [Pot: 6]
Player 3 (position 3): bets/raises to 8 [Pot: 14]
Player 4 (position 4): calls 8 [Pot: 22]
Player 0 (position 0): calls 6 [Pot: 28]
Player 1 (position 1): folds [Pot: 28